# ⚡️⚡️⚡️  ruDALLE torch lightning finetuning notebook ⚡️⚡️⚡️

In [1]:
%%capture
!pip install rudalle==1.1.0
!pip install bitsandbytes-cuda113 > /dev/null
!pip install wandb > /dev/null
!pip install pytorch-lightning > /dev/null
!pip install deepspeed > /dev/nul
!pip install gdown > /dev/null

In [3]:
#@markdown Lets download data

import gdown
url = "http://drive.google.com/uc?id=17bPt7G3N_vGKCCxppIOPbPlhv1qUnv0o"
output = "food.zip"
gdown.download(url, output, quiet=False)

Downloading...
From: http://drive.google.com/uc?id=17bPt7G3N_vGKCCxppIOPbPlhv1qUnv0o
To: /workspace/food.zip
100%|██████████| 34.8M/34.8M [00:00<00:00, 63.6MB/s]


'food.zip'

In [4]:
!unzip food.zip > /dev/null

food/Булочки с начинкой «Молочная нежность».jpg:  mismatching "local" filename (food/╨С╤Г╨╗╨╛╤З╨║╨╕ ╤Б ╨╜╨░╤З╨╕╨╜╨║╨╛╨╣ ┬л╨Ь╨╛╨╗╨╛╤З╨╜╨░╤П ╨╜╨╡╨╢╨╜╨╛╤Б╤В╤М┬╗.jpg),
         continuing with "central" filename version
food/Запеченный картофель «Чесночно-сырный».jpg:  mismatching "local" filename (food/╨Ч╨░╨┐╨╡╤З╨╡╨╜╨╜╤Л╨╣ ╨║╨░╤А╤В╨╛╤Д╨╡╨╗╤М ┬л╨з╨╡╤Б╨╜╨╛╤З╨╜╨╛-╤Б╤Л╤А╨╜╤Л╨╣┬╗.jpg),
         continuing with "central" filename version
food/Лепешки «В путь дорожку!».jpg:  mismatching "local" filename (food/╨Ы╨╡╨┐╨╡╤И╨║╨╕ ┬л╨Т ╨┐╤Г╤В╤М ╨┤╨╛╤А╨╛╨╢╨║╤Г!┬╗.jpg),
         continuing with "central" filename version
food/Салат «Анастасия».jpg:  mismatching "local" filename (food/╨б╨░╨╗╨░╤В ┬л╨Р╨╜╨░╤Б╤В╨░╤Б╨╕╤П┬╗.jpg),
         continuing with "central" filename version
food/Салат «Коко Шанель».jpg:  mismatching "local" filename (food/╨б╨░╨╗╨░╤В ┬л╨Ъ╨╛╨║╨╛ ╨и╨░╨╜╨╡╨╗╤М┬╗.jpg),
         continuing with "central" filename version
food/Салат «Курица в шубе».jpg:  mismatching "local" filename (food/╨б╨░╨

For better results use data with good text descritption, and propetis what you want to train model.
For example - if you want to teach model to generate shape provide text input such as: "a thing in shape of circle" 'circle.png'

In [8]:
import os
import sys
import random
from collections import Counter

import PIL
import torch
import numpy as np
import pandas as pd
import bitsandbytes as bnb
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from tqdm import tqdm

from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from rudalle import get_tokenizer, get_vae
from rudalle.utils import seed_everything
import pytorch_lightning as pl



In [9]:

from rudalle.pipelines import generate_images, show, super_resolution
from rudalle import get_tokenizer, get_vae, get_realesrgan,get_rudalle_model
from rudalle.utils import seed_everything
device = 'cuda'
model = get_rudalle_model('Malevich', pretrained=True, fp16=True, device='cpu')
vae = get_vae().to(device)
tokenizer = get_tokenizer()

Warning! Using both fp16 and cpu doesnt support. You can use cuda device or turn off fp16.


/opt/conda/lib/python3.7/site-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

◼️ Malevich is 1.3 billion params model from the family GPT3-like, that uses Russian language and text+image multi-modality.
Working with z of shape (1, 256, 32, 32) = 262144 dimensions.


Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

vae --> ready


Downloading:   0%|          | 0.00/224k [00:00<?, ?B/s]

tokenizer --> ready


In [10]:
1024+128

1152

In [11]:
class Args():
    def __init__(self):
        self.epochs = 2
        self.bs = 1
        self.save_every = 100 
        self.lr = 4e-5



        self.text_seq_length = 128
        self.total_seq_length = 1152
        self.save_path='checkpoints'
        self.model_name = 'awesomemodel_'
        self.prefix_length = 10
        self.image_size = 256
        self.freeze = False
        self.clip = 0.24
        self.warmup_steps =50
        
        self.wandb = True
args = Args()
if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)

In [12]:
class FoodDataset(Dataset):
    def __init__(self, file_path, csv_path, tokenizer, shuffle=True):
        self.tokenizer = tokenizer
        self.samples = []
        self.image_transform = T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.RandomResizedCrop(args.image_size, scale=(1., 1.), ratio=(1., 1.)),
            T.ToTensor()
        ])

        df = pd.read_csv(csv_path)
        df.columns = ['index', 'belok', 'fats', 'uglevod', 'kkal', 'name', 'path']

        for belok, fats, uglevod, kkal, caption, f_path in zip(
            df['belok'],df['fats'], df['uglevod'], df['kkal'], df['name'], df['path']
        ):
            caption = f'блюдо: {caption}; белков: {belok}; жиров: {fats}; углеводов: {uglevod}; ккал: {kkal};'
            if len(caption)>10 and len(caption)<100 and os.path.isfile(f'{file_path}/{f_path}'):
                self.samples.append([file_path, f_path, caption.lower()])
        if shuffle:
            np.random.shuffle(self.samples)
            print('Shuffled')

    def __len__(self):
        return len(self.samples)

    def load_image(self, file_path, img_name):
        return PIL.Image.open(f'{file_path}/{img_name}')

    def __getitem__(self, item):
        item = item % len(self.samples)
        file_path, img_name, text = self.samples[item]

        try:
            image = self.load_image(file_path, img_name)
            image = self.image_transform(image)
        except Exception as err: 
            print(err)
            random_item = random.randint(0, len(self.samples) - 1)
            return self.__getitem__(random_item)
        
        text = text.lower().strip()
        encoded = self.tokenizer.encode_text(text, text_seq_length=args.text_seq_length)       
        return encoded, image

In [13]:
from torch.utils.data import Dataset, DataLoader

class FoodDataModule(pl.LightningDataModule):
    
  def __init__(self, file_path, csv_path, tokenizer):
    super().__init__()
    self.file_path = file_path
    self.csv_path = csv_path

  def setup(self, stage=None):
    self.train_dataset = FoodDataset(file_path= self.file_path, 
                                     csv_path = self.csv_path, 
                                     tokenizer = tokenizer)

    

  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=args.bs,
      shuffle=True,
      
    )
dataset = FoodDataset(file_path = 'food' ,csv_path = 'food/food.csv',tokenizer = tokenizer)
args.train_steps = len(dataset)//args.bs
print(args.train_steps )


Shuffled
474


In [14]:
from transformers import AdamW, get_linear_schedule_with_warmup
import bitsandbytes as bnb



def freeze(params,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=False):
    
    for name, p in params.named_parameters():        
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
           
    return params


class Rudalle_(pl.LightningModule):
    
  def __init__(self, args, vae):
    super().__init__()
    
    
    print(self.device)
    self.model = get_rudalle_model('Malevich', pretrained=True, fp16=True, device='cuda')

    if args.freeze:
        self.model = freeze(self.model) 
    #self.vae = get_vae(dwt=False).to(self.device)

   
    print(self.device)
  
  def forward(self, 
            input_ids, 
            return_loss=True):
   
    input_ids = input_ids.to(self.device)
    
    attention_mask = torch.tril(torch.ones((args.bs, 1, args.total_seq_length, args.total_seq_length)))
    attention_mask = attention_mask.to(self.device)
    
    loss, loss_values = self.model.forward(input_ids, 
                                           attention_mask, 
                                           return_loss=return_loss)
    
    return loss



  def training_step(self, batch):
    text, image = batch[0], batch[1]

    text = text.to(self.device)
    image = image.to(self.device)

    image_input_ids = vae.get_codebook_indices(image)

    image_input_ids = image_input_ids.to(self.device)
    
    input_ids = torch.cat((text, image_input_ids), dim=1)
    
    loss = self.forward(input_ids, return_loss=True)
    
    
    self.log("train_loss", loss, prog_bar=True, logger=True)

    return {"loss": loss}

  def training_epoch_end(self, outputs):
    pass

  def configure_optimizers(self):
    
    optimizer = AdamW(self.parameters(), lr=args.lr, betas=(0.9, 0.995))

    

   
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
      optimizer, 
      max_lr=args.lr,
      final_div_factor=500, 
      steps_per_epoch=args.train_steps,
      epochs=args.epochs 
    )

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )
      

In [15]:
from pytorch_lightning.loggers import WandbLogger
# я использую wandb в качестве логера, если надо замените на тенсорборду
# i prefer wandb, change to tensobord if u want
wandb_logger = WandbLogger(project="rudalle")

wandb: Currently logged in as: alexwortega. Use `wandb login --relogin` to force relogin


In [16]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
checkpoint_callback = ModelCheckpoint(
  dirpath=args.save_path,
  filename=args.model_name,
  save_top_k=1,
  verbose=True,
  mode="min"
)


In [19]:
vae.to('cuda')
model = Rudalle_(args, vae)

data_module = FoodDataModule(file_path='food', csv_path ='food/food.csv', tokenizer=tokenizer)

trainer = pl.Trainer(
  logger=wandb_logger,
  checkpoint_callback=checkpoint_callback,
  max_epochs=2,
  
  accelerator="gpu",
  accumulate_grad_batches=5,
  devices=[0],
  progress_bar_refresh_rate=30
)

cpu


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


◼️ Malevich is 1.3 billion params model from the family GPT3-like, that uses Russian language and text+image multi-modality.
cpu


In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | FP16Module | 1.3 B 
-------------------------------------
1.3 B     Trainable params
0         Non-trainable params
1.3 B     Total params
5,241.644 Total estimated model params size (MB)


Shuffled


Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1366: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  "Positional args are being deprecated, use kwargs instead. Refer to "


In [ ]:
trainer.save_checkpoint('/rudalle')
def _fix_pl(path):
  d = torch.load(path)["state_dict"]
  checkpoint = {}
  for key in d.keys():
    checkpoint[key.replace('model.','')] = d[key]
  torch.save(checkpoint,'fixed.pt')
_fix_pl('rudalle')

# deepspeed (advanced)

In [ ]:
%%writefile ds_config.json

{
  "train_batch_size": 15,
  "fp16": {
    "enabled": true,
    "min_loss_scale": 1,
    "opt_level": "O3"
  },
  "zero_optimization": {
    "stage": 2,
    "offload_param": {
      "device": "nvme",
      "nvme_path": "/home/asta/deepspeed",
      "buffer_count": 5,
      "buffer_size": 1e8,
      "reduce_bucket_size":2e7,
      "max_in_cpu": 1e9
    },
    "offload_optimizer": {
      "device": "nvme",
      "nvme_path": "/home/asta/deepspeed",
      "buffer_count": 4,
      "pipeline_read": false,
      "pipeline_write": false,
      "pin_memory": true
    },
    "allgather_partitions": true,
    "allgather_bucket_size": 5e8,
    "contiguous_gradients": true,
    "overlap_comm": true,
    "aio": {
      "block_size": 1048576,
      "queue_depth": 8,
      "thread_count": 1,
      "single_submit": false,
      "overlap_events": true
    }
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 5e-05,
      "betas": [
        0.9,
        0.999
      ],
      "eps": 1e-08
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 5e-05,
      "warmup_num_steps": 100
    }
  }
}

In [ ]:
%%writefile train.py
import os
import sys
import random
from collections import Counter

import PIL
import torch
import numpy as np
import pandas as pd
import bitsandbytes as bnb
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from tqdm import tqdm

from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from rudalle import get_tokenizer, get_vae
from rudalle.utils import seed_everything
import pytorch_lightning as pl
from pytorch_lightning.plugins import DeepSpeedPlugin


from rudalle.pipelines import generate_images, show, super_resolution
from rudalle import get_tokenizer, get_vae, get_realesrgan,get_rudalle_model
from rudalle.utils import seed_everything
device = 'cuda'
model = get_rudalle_model('Malevich', pretrained=True, fp16=True, device='cpu')
vae = get_vae().to(device)
tokenizer = get_tokenizer()

class Args():
    def __init__(self):
        self.epochs = 2
        self.bs = 1
        self.save_every = 100 
        self.lr = 4e-5



        self.text_seq_length = 128
        self.total_seq_length = 1152
        self.save_path='checkpoints'
        self.model_name = 'awesomemodel_'
        self.prefix_length = 10
        self.image_size = 256
        self.freeze = False
        self.clip = 0.24
        self.warmup_steps =50
        
        self.wandb = True
args = Args()
if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)

class FoodDataset(Dataset):
    def __init__(self, file_path, csv_path, tokenizer, shuffle=True):
        self.tokenizer = tokenizer
        self.samples = []
        self.image_transform = T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.RandomResizedCrop(args.image_size, scale=(1., 1.), ratio=(1., 1.)),
            T.ToTensor()
        ])

        df = pd.read_csv(csv_path)
        df.columns = ['index', 'belok', 'fats', 'uglevod', 'kkal', 'name', 'path']

        for belok, fats, uglevod, kkal, caption, f_path in zip(
            df['belok'],df['fats'], df['uglevod'], df['kkal'], df['name'], df['path']
        ):
            caption = f'блюдо: {caption}; белков: {belok}; жиров: {fats}; углеводов: {uglevod}; ккал: {kkal};'
            if len(caption)>10 and len(caption)<100 and os.path.isfile(f'{file_path}/{f_path}'):
                self.samples.append([file_path, f_path, caption.lower()])
        if shuffle:
            np.random.shuffle(self.samples)
            print('Shuffled')

    def __len__(self):
        return len(self.samples)

    def load_image(self, file_path, img_name):
        return PIL.Image.open(f'{file_path}/{img_name}')

    def __getitem__(self, item):
        item = item % len(self.samples)
        file_path, img_name, text = self.samples[item]

        try:
            image = self.load_image(file_path, img_name)
            image = self.image_transform(image)
        except Exception as err: 
            print(err)
            random_item = random.randint(0, len(self.samples) - 1)
            return self.__getitem__(random_item)
        
        text = text.lower().strip()
        encoded = self.tokenizer.encode_text(text, text_seq_length=args.text_seq_length)       
        return encoded, image
from torch.utils.data import Dataset, DataLoader

class FoodDataModule(pl.LightningDataModule):
    
  def __init__(self, file_path, csv_path, tokenizer):
    super().__init__()
    self.file_path = file_path
    self.csv_path = csv_path

  def setup(self, stage=None):
    self.train_dataset = FoodDataset(file_path= self.file_path, 
                                     csv_path = self.csv_path, 
                                     tokenizer = tokenizer)

    

  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=args.bs,
      shuffle=True,
      
    )
dataset = FoodDataset(file_path = 'food' ,csv_path = 'food/food.csv',tokenizer = tokenizer)
args.train_steps = len(dataset)//args.bs
print(args.train_steps )

from transformers import AdamW, get_linear_schedule_with_warmup
import bitsandbytes as bnb



def freeze(params,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=False):
    
    for name, p in params.named_parameters():        
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
           
    return params


class Rudalle_(pl.LightningModule):
    
  def __init__(self, args, vae):
    super().__init__()
    
    
    print(self.device)
    self.model = get_rudalle_model('Malevich', pretrained=True, fp16=True, device='cuda')

    if args.freeze:
        self.model = freeze(self.model) 
    #self.vae = get_vae(dwt=False).to(self.device)

   
    print(self.device)
  
  def forward(self, 
            input_ids, 
            return_loss=True):
   
    input_ids = input_ids.to(self.device)
    
    attention_mask = torch.tril(torch.ones((args.bs, 1, args.total_seq_length, args.total_seq_length)))
    attention_mask = attention_mask.to(self.device)
    
    loss, loss_values = self.model.forward(input_ids, 
                                           attention_mask, 
                                           return_loss=return_loss)
    
    return loss



  def training_step(self, batch):
    text, image = batch[0], batch[1]

    text = text.to(self.device)
    image = image.to(self.device)

    image_input_ids = vae.get_codebook_indices(image)

    image_input_ids = image_input_ids.to(self.device)
    
    input_ids = torch.cat((text, image_input_ids), dim=1)
    
    loss = self.forward(input_ids, return_loss=True)
    
    
    self.log("train_loss", loss, prog_bar=True, logger=True)

    return {"loss": loss}

  def training_epoch_end(self, outputs):
    pass

  def configure_optimizers(self):
    
    optimizer = bnb.optim.Adam8bit(self.parameters(), lr=args.lr, betas=(0.9, 0.995))

    

   
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
      optimizer, 
      max_lr=args.lr,
      final_div_factor=500, 
      steps_per_epoch=args.train_steps,
      epochs=args.epochs 
    )

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )
      


from pytorch_lightning.loggers import WandbLogger
# я использую wandb в качестве логера, если надо замените на тенсорборду
# i prefer wandb, change to tensobord if u want
wandb_logger = WandbLogger(project="rudalle")

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
checkpoint_callback = ModelCheckpoint(
  dirpath=args.save_path,
  filename=args.model_name,
  save_top_k=1,
  verbose=True,
  mode="min"
)

vae.to('cuda')
model = Rudalle_(args, vae)

data_module = FoodDataModule(file_path='food', csv_path ='food/food.csv', tokenizer=tokenizer)




if __name__ == "__main__":


  trainer = pl.Trainer(
    logger=wandb_logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=2,
    #precision=16,
   
    strategy=DeepSpeedPlugin("/content/ds_config.json"),
    
    
    progress_bar_refresh_rate=30
  )

  trainer.fit(model, data_module)


In [ ]:
!python3 train.py